# 卫星图片分类

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pathlib
%matplotlib inline

#### 读取数据

In [3]:
# 数据目录
data_dir = "dataset/satellite_picture"

data_root = pathlib.Path(data_dir)
# 所有文件的路径
all_image_paths = list(data_root.glob('*/*'))
all_image_paths[:3]

[WindowsPath('dataset/satellite_picture/airplane/airplane_001.jpg'),
 WindowsPath('dataset/satellite_picture/airplane/airplane_002.jpg'),
 WindowsPath('dataset/satellite_picture/airplane/airplane_003.jpg')]

In [4]:
# 转为字符串
all_image_paths = [str(path) for path in all_image_paths]
all_image_paths[:3]

['dataset\\satellite_picture\\airplane\\airplane_001.jpg',
 'dataset\\satellite_picture\\airplane\\airplane_002.jpg',
 'dataset\\satellite_picture\\airplane\\airplane_003.jpg']

In [5]:
# 数据集总数
image_num = len(all_image_paths)
image_num

1400

#### 创建训练集

In [6]:
# 加载图片的函数
def load_one_image(path):
    raw_img = keras.preprocessing.image.load_img(path, target_size=(256, 256))
    img = keras.preprocessing.image.img_to_array(raw_img)
    img = img / 255.0
    #plt.imshow(img)
    return img
#load_one_image(all_image_paths[0])

In [7]:
def load_images(image_path_list):
    image_dataset = []
    for path in image_path_list:
        image_dataset.append(load_one_image(path))
    return np.array(image_dataset)
image_dataset = load_images(all_image_paths)
image_dataset.shape

(1400, 256, 256, 3)

#### 创建标签

In [8]:
# 标签的名字
label_names = sorted([item.name for item in data_root.glob('*/')])
label_names

['airplane', 'lake']

In [9]:
# 标签名字到标签序号
name_to_index = dict((name, index) for index, name in enumerate(label_names))
name_to_index

{'airplane': 0, 'lake': 1}

In [10]:
label_dataset = [name_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
label_dataset[0:3]

[0, 0, 0]

#### 构建tf.data对象

In [11]:
image_dataset = tf.data.Dataset.from_tensor_slices(image_dataset)
image_dataset

<TensorSliceDataset shapes: (256, 256, 3), types: tf.float32>

In [12]:
label_dataset = tf.data.Dataset.from_tensor_slices(label_dataset)
label_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [13]:
dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
dataset

<ZipDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>

#### 分离训练集和验证集

In [14]:
dataset = dataset.shuffle(image_num)

In [15]:
validation_num = int(image_num * 0.2)
train_num = image_num - validation_num
print("train_num:", train_num)
print("validation_num:", validation_num)
train_data = dataset.skip(validation_num)
validation_data = dataset.take(validation_num)
print("train_data:", train_data)
print("validation_data:", validation_data)

train_num: 1120
validation_num: 280
train_data: <SkipDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>
validation_data: <TakeDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>


In [16]:
BATCH_SIZE = 32
train_data = train_data.shuffle(train_num).repeat().batch(BATCH_SIZE)
validation_data = validation_data.batch(BATCH_SIZE)

#### 构建模型训练

In [17]:
model = tf.keras.Sequential()   #顺序模型
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 122, 122, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 59, 59, 256)       2

In [18]:
model.compile(optimizer = keras.optimizers.Adam(),
              loss = "binary_crossentropy",
              metrics = ["accuracy"]
)

In [ ]:
train_steps = train_num // BATCH_SIZE
val_steps = validation_num // BATCH_SIZE
train_history = model.fit(train_data,
                          epochs = 10,
                          steps_per_epoch = train_steps,
                          validation_data = validation_data,
                          validation_steps = val_steps
)

Train for 35 steps, validate for 8 steps
Epoch 1/10
35/35 [==============================] - 45s 1s/step - loss: 0.9537 - accuracy: 0.4982 - val_loss: 0.3961 - val_accuracy: 0.5312
Epoch 2/10
35/35 [==============================] - 35s 988ms/step - loss: 0.4810 - accuracy: 0.8062 - val_loss: 0.3518 - val_accuracy: 0.9727
Epoch 3/10
35/35 [==============================] - 34s 972ms/step - loss: 0.2222 - accuracy: 0.9482 - val_loss: 0.1761 - val_accuracy: 0.9453
Epoch 4/10
35/35 [==============================] - 35s 991ms/step - loss: 0.1500 - accuracy: 0.9580 - val_loss: 0.1062 - val_accuracy: 0.9648
Epoch 5/10
35/35 [==============================] - 37s 1s/step - loss: 0.1196 - accuracy: 0.9634 - val_loss: 0.1262 - val_accuracy: 0.9492
Epoch 6/10
35/35 [==============================] - 34s 971ms/step - loss: 0.1362 - accuracy: 0.9643 - val_loss: 0.0747 - val_accuracy: 0.9727
Epoch 7/10
35/35 [==============================] - 34s 972ms/step - loss: 0.1458 - accuracy: 0.9616 - val_

#### 打印训练图线

In [ ]:
train_history.history.keys()

In [ ]:
train_loss = train_history.history["loss"]
train_acc = train_history.history["accuracy"]
val_loss = train_history.history["val_loss"]
val_acc = train_history.history["val_accuracy"]

plt.plot(train_history.epoch, train_loss, label='train_loss')
plt.plot(train_history.epoch, val_loss, label='val_loss')
plt.legend()

In [ ]:
plt.plot(train_history.epoch, train_acc, label='train_acc')
plt.plot(train_history.epoch, val_acc, label='val_acc')
plt.legend()